In [75]:
import time
import random
from collections import deque

# Ucitavanje grafa

In [76]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            # Dodavanje suseda
            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph


# Funkcije za racunanje betweenness centralnosti

In [77]:
def bfs_shortest_paths(graf, start):
    dist = {node: float('inf') for node in graf} 
    dist[start] = 0
    queue = deque([start])
    paths = {node: [] for node in graf}
    paths[start] = [start]
    
    while queue:
        current = queue.popleft()
        
        for neighbor in graf[current]:
            if dist[neighbor] == float('inf'):
                dist[neighbor] = dist[current] + 1
                queue.append(neighbor)
            
            if dist[neighbor] == dist[current] + 1:
                paths[neighbor].append(current)
    
    return dist, paths

def calculate_betweenness(graf):
    betweenness = {node: 0 for node in graf}
    
    for start in graf:
        dist, paths = bfs_shortest_paths(graf, start)
        
        for node in graf:
            if node == start:
                continue
            
            for neighbor in paths[node]:
                if neighbor != start:
                    betweenness[neighbor] += 1
    
    return betweenness

def betweenness_centralnost(cvor, betweenness):
    return betweenness.get(cvor, 0) 

# Pohlepni algoritam

In [78]:
def greedy_burning(graf):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graf.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa
    
    # ako imamo vise kandidata, biramo onog sa najvisim betweenness
    
    najveci_stepen = max(len(graf[node]) for node in graf)
    kandidati = [node for node in graf if len(graf[node]) == najveci_stepen]
    trenutni_cvor = None
    
    if len(kandidati) > 1:
        betweenness = calculate_betweenness(graf)
        trenutni_cvor = max(kandidati, key=lambda node: betweenness_centralnost(node, betweenness))
    else:
        trenutni_cvor = kandidati[0]
        
    S.append(trenutni_cvor)
    B.add(trenutni_cvor)
    NB.remove(trenutni_cvor)
  
    
    
    while len(B)<len(graf):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        
        # Provjera: ako su svi cvorovi u NB zapravo cvorovi koji ce biti zapaljeni u narednom koraku (jer su direktni susjedi
        # zapaljenom cvoru nekom), onda cemo izabrati neki random cvor iz NB cisto kao simbol jos jednog koraka
        
        FB = []
        for b in B:
            susjedi_b1 = graf[b]
            for susjed in susjedi_b1:
                if susjed in NB:
                    if susjed not in FB:
                        FB.append(susjed)
         
        if set(FB) == NB:
            random_cvor = random.choice(list(NB))
            S.append(random_cvor)
            
        
        novozapaljeni = []
        for cvor in B:
            susjedi = graf[cvor]  # svi susjedi trenutnog zapaljenog cvora
            for susjed in susjedi:
                if susjed in NB:  # ako susjed nije zapaljen
                    novozapaljeni.append(susjed)  # dodajemo ga u zapaljene
                    NB.remove(susjed)  
        
        for nz in novozapaljeni:
            B.add(nz) #dodajemo sve susjede u zapaljene
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        
        FB2 = []
        for b in B:
            susjedi_b = graf[b]
            for susjed in susjedi_b:
                if susjed in NB:
                    if susjed not in FB2:
                        FB2.append(susjed)
        
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_broj_zapaljenih = -1
        najbolji_cvor = None
        kandidati_za_najboljeg = []
        
        if NB:
            for cvor in NB:
                if cvor not in FB2: 
                    nezapaljeni_susjedi = len([n for n in graf[cvor] if n not in B and n not in FB2])
                    if nezapaljeni_susjedi>max_broj_zapaljenih:
                        max_broj_zapaljenih = nezapaljeni_susjedi
                        
            for cvor in NB:
                if cvor not in FB2:
                    nezapaljeni_susjedi = len([n for n in graf[cvor] if n not in B and n not in FB2])
                    if nezapaljeni_susjedi == max_broj_zapaljenih:
                        kandidati_za_najboljeg.append(cvor) # u slucaju da imamo vise kandidata za najbolji cvor
                        
            
            if len(kandidati_za_najboljeg)>1:# izbor na osnovu betweenness centralnosti
                 najbolji_cvor = max(kandidati_za_najboljeg, key=lambda cvor: betweenness_centralnost(cvor, calculate_betweenness(graf)))
            elif len(kandidati_za_najboljeg)==1:
                najbolji_cvor = kandidati_za_najboljeg[0]
                
            if najbolji_cvor is not None:
                S.append(najbolji_cvor)
                B.add(najbolji_cvor)
                NB.remove(najbolji_cvor)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_cvor = random.choice(list(NB))
                S.append(random_cvor)
                B.add(random_cvor)
                NB.remove(random_cvor)
        else:
            break
        
        
        
    return S
        

# ILS

In [79]:

def coverage_score(graph, node, burned_nodes):
    """
    Merenje pokrivenosti: Broj novih čvorova koje bi čvor 'node' pokrio.
    """
    # Broj suseda koji nisu već u paljenju
    neighbors = set(graph[node]) - burned_nodes
    return len(neighbors)

def evaluate_sequence(graph, S):
    """
    Evaluacija dužine sekvence potrebne za paljenje celog grafa.
    """
    burned_nodes = set()
    for step, node in enumerate(S):
        burned_nodes.add(node)
        # Širi paljenje na susede
        neighbors = set(graph[node]) - burned_nodes
        burned_nodes.update(neighbors)
        
        if len(burned_nodes) == len(graph):
            return step + 1
    
    return len(S)  # Ako nije sve pokriveno, vraća dužinu sekvence

def perturb_sequence(graph, S, burned_nodes):
    """
    Perturbira sekvencu paljenja pokušajem zamene prvog čvora sa čvorom koji ima bolju pokrivenost.
    """
    best_score = -1
    best_node = None

    # Zamenjujemo prvi čvor sa čvorom koji ima bolju pokrivenost
    for node in graph:
        if node not in S:  # Ne menjaš čvorove koji su već u sekvenci
            score = coverage_score(graph, node, set(S))
            if score > best_score:
                best_score = score
                best_node = node

    if best_node is not None:
        # Zamenjujemo prvi čvor sa onim sa najboljim score-om
        S[0] = best_node
    
    return S

def ils_graph_burning(graph, max_iterations=500):
    """
    Iterated Local Search za Graph Burning Problem.
    """
    # Generiši početno rešenje korišćenjem greedy algoritma
    S = greedy_burning(graph)
    best_S = S[:]
    best_length = evaluate_sequence(graph, best_S)

    for _ in range(max_iterations):
        # Perturbiraj sekvencu
        S_perturbed = perturb_sequence(graph, S[:], set(best_S))

        # Evaluiraj perturpirano rešenje
        length = evaluate_sequence(graph, S_perturbed)

        # Ažuriraj najbolje rešenje ako je poboljšano
        if length < best_length:
            best_S = S_perturbed[:]
            best_length = length

    return best_S, best_length

In [82]:
startTime = time.time()
#filename= "word.mtx"  
filename = "grafovi/manji/DD68.mtx"

graph = load_graph_from_mtx(filename)
burn_order = greedy_burning(graph)
ils = ils_graph_burning(graph)

print("Redoslijed paljenja cvorova:", burn_order)
print("b(G)=",len(burn_order))

print("Redoslijed paljenja cvorova:", ils)
print("b(G)=",ils)
print("--- %s sekundi ---" % (time.time() - startTime))


Redoslijed paljenja cvorova: [7, 35, 259, 307, 280, 289, 625, 331, 91, 165, 221, 612, 412, 237]
b(G)= 14
Redoslijed paljenja cvorova: ([7, 35, 259, 307, 280, 289, 625, 331, 91, 165, 221, 612, 412, 237], 14)
b(G)= ([7, 35, 259, 307, 280, 289, 625, 331, 91, 165, 221, 612, 412, 237], 14)
--- 64.06750559806824 sekundi ---
